In [1]:
# Quelques imports utiles
import random
import utils

## Lecture du corpus

In [2]:
import random
from utils import preprocess

with open("data/Les_Misérables.txt", "r", encoding="utf8") as f:
    corpus = f.read()

max_sentences = 1000
corpus = preprocess(corpus)[:max_sentences]

for doc in random.sample(corpus, 5):
    print(*doc)

il passa toute la journée auprès de lui oubliant la nourriture et le sommeil priant dieu pour l âme du condamné et priant le condamné pour la sienne propre
dans cette obéissance là il peut encore y avoir de la faute mais la faute ainsi faite est vénielle
liquidation totale
au néant
nous ferons comme eux et nous le nommerons ainsi dans l occasion


In [3]:
# Quelques 2-grammes
bigrams = [line.split() for line in 
"""<s> <s>
cosette était
elle était
je suis
sombre imbécile
c est""".split("\n")]

trigrams = [line.split() for line in 
"""<s> <s> il
cosette était sortie
elle était toujours
je suis fort
sombre imbécile couac
c est un""".split("\n")]

## Maximum de vraisemblance

L'estimateur à maximum de vraisemblance EMV (en anglais : MLE, pour *Maximum Likelihood Estimator*).

Soit $D = w_1 \ldots w_N$ un corpus composés de $N$ mots. Modèle de langue $n$-gramme : soit $n \geq 1$, et $w_i$ un mot. Son contexte est composé des $n-1$ mots qui le précèdent : $c_i = w_{i-n+1}, \ldots, w_{i-1}$.

$$
P_{MLE}(w_i | c_i) = \frac{|c_i, w_i|}{|c_i|}
$$

In [4]:
from models.mle import MLE

mle = MLE(corpus, 3)

for *context, word in trigrams:
    print("P", word, "|", *context, "=", mle.proba(word, context))
    
print("\n")

for context in bigrams:
    print(*context, "->", mle.predict_next(context))
    
print("Most frequent n-grams:")
for n in (2, 3):
    print("\nn =", n)
    for ngram, count in mle.counts[n].most_common(5):
        print(*ngram, count)

P il | <s> <s> = 0.133
P sortie | cosette était = 0.0
P toujours | elle était = 0.0
P fort | je suis = 0.0
P couac | sombre imbécile = 0.0
P un | c est = 0.09523809523809523


<s> <s> -> elles
cosette était -> rire
elle était -> atteinte
je suis -> un
sombre imbécile -> rire
c est -> sa
Most frequent n-grams:

n = 2
<s> <s> 1000
<s> il 133
l évêque 94
de la 70
qu il 58

n = 3
<s> <s> il 133
<s> <s> l 56
<s> <s> le 51
<s> l évêque 35
<s> <s> c 34


In [5]:
utils.inspect(mle.proba)

```python
    def proba(self, word: str, context: List[str]):
        p_context = self.counts[context]
        if p_context:
            return self.counts[(*context, word)] / p_context
        return 0.0
```

In [6]:
for _ in range(5):
    print(*mle.generate())
    print("-")

<s> <s> c était un malheureux </s>
-
<s> <s> ils n ont pas de voisins pas même devant un refus et il disait cela ne me laisse pas enguirlander par des questions d argent et une fenêtre et enfin trois cent quarante mille cabanes qui n avait de preuves que contre elle </s>
-
<s> <s> vous faites bien de venir regarder un homme en cheveux blancs qui souriait au soleil </s>
-
<s> <s> il faut qu il se parlait à lui </s>
-
<s> <s> le cardinal </s>
-


## Good-Turing


$$
r^* = (r + 1) \times \frac{n_{r+1}}{n_r}
$$